In [1]:
import numpy, pandas, re
from collections import defaultdict
from math import ceil, floor, comb, isnan

In [2]:
wep = { 'attacks': '3', 'bsws': 3, 'strength': 5, 'AP': -2, 'damage': '2', 'kws': ['lethal hits', 'devastating wounds', 'sustained hits 2'] }
target = { 'toughness': 3, 'save': 2, 'invuln': 4, 'wounds': 3, 'kws':['infantry'], 'abilities': {'feel no pain': 5, 'stealth': True} }

In [3]:
# Helper functions for damage distributions

def convolve(d1,d2, n_wounds=None):
    res = defaultdict(lambda: 0)
    if n_wounds is None:
        for k1,v1 in d1.items():
            for k2,v2 in d2.items():
                res[k1+k2] += v1*v2
    else: # Limit damage to n_wounds units
        for k1,v1 in d1.items():
            for k2,v2 in d2.items():
                k1m, k2m = k1%n_wounds, k2%n_wounds
                if k1m + k2m > n_wounds: res[k1+k2-(k1m+k2m)%n_wounds] += v1*v2
                else: res[k1+k2] += v1*v2
    return res

def mult_ddist_vals(d, val):
    res = defaultdict(lambda: 0)
    for k1,v1 in d.items():
        res[int(ceil(k1*val))] += v1 
    return res

def mult_ddist_probs(d,p):
    return { k:v*p for k,v in d.items() }

def threshold_ddist(dd,val,lt=True):
    for k in list(dd.keys()): 
        if (lt and k<val) or (not lt and k>val): 
            dd[val]+=dd[k]
            del dd[k]

def add_ddist(d1,d2):
    res = defaultdict(lambda: 0)
    for k1,v1 in d1.items():
        res[k1] += v1  
    for k2,v2 in d2.items():
        res[k2] += v2 
    return res
    
def flatdist(n):
    return { (i+1):1/n for i in range(n) }

# q is the probability of saving the damage
def fnp_transform(d, q):
    p = 1.0-q
    res = defaultdict(lambda: 0)
    for k,v in d.items():
        for i in range(k+1): # Binomial distribution
            res[i] += v*comb(k,i)*(p**i)*(q**(k-i))
    return res

# Prune all values with prob below ratio * <max prob>
def dd_prune(d, ratio):
    t = ratio*max(d.values())
    return { k:v for k,v in d.items() if v>t }

def dd_mean(dd):
    val = 0.0
    for k, v in dd.items():
        val += k*v
    return val

def dd_max(dd):
    return max(dd.keys())

def dd_psum(dd):
    return sum(dd.values())


In [4]:
def dd_from_str(dstr):
    dd = { 0: 1.0 }
    for t in dstr.split('+'):
        if 'D' in t:
            d = t.split('D')
            if d[0]=='': d = [1,int(d[1])]
            else: d = list(map(int,d))
            for _ in range(d[0]):
                nd = flatdist(d[1])
                dd = convolve(dd,nd)
        else:
            d = int(t)
            dd = convolve(dd,{d:1.0})
    return dd

In [ ]:
def single_dam_dist(dmgstr, n_wounds=None, div=1,mult=1,add=0,fnp=None):

    # Create dmgstr distribution
    dd = dd_from_str(dmgstr)

    # Apply simple modifiers
    if div!=1: dd = mult_ddist_vals(dd,1.0/div)
    if mult!=1: dd = mult_ddist_probs(dd,mult)
    if add!=0: dd = convolve(dd,{add:1.0})

    # Threshold to 1
    threshold_ddist(dd,1,True)

    # Apply FNP
    if fnp: dd = fnp_transform(dd,(7-fnp)/6)

    # Threshold to n_wounds
    if n_wounds: threshold_ddist(dd,n_wounds,False)

    #print(dd_mean(dd))

    return dd
    
single_dam_dist('2D6+2',n_wounds=2,div=2, add=-2,fnp=4)#None)

In [6]:
def get_hit_probs(wep):
    # Prob to hit
    if 'torrent' in wep['kws']:
        p_hcrit, p_hit = 0, 1
    else:
        hit_crit = 6 # Normally crit on 6+ but can be something else
        p_hcrit = (7-hit_crit)/6.0
        p_hit = max((6*p_hcrit),min(5, # hit_crit always hits, 1 always misses
                    (7-wep['bsws'])))/6.0
    return p_hit,p_hcrit

In [29]:
def atk_success_prob(wep,target,crit_hit=None, verbose=False):

    # TODO:
    # Cover / Ignores cover
    # Rerolls (1 and all) + fish-for-sixes if better.
    # ANTI keywords

    # Probs to hit
    p_hit, p_hcrit = get_hit_probs(wep)
        
    # Check if fn parameter already tells us if it was a crit or not
    # This behavior is needed for Sustained hits as they also affect hit counts
    if crit_hit: p_hcrit,p_hit = 1,1
    elif crit_hit==False: p_hcrit,p_hit = 0, 1

    if verbose: print("Hit",p_hit,p_hcrit)

    # Prob to wound

    wound_crit = 6 # Anti-keywords change this
    p_wcrit = (7-wound_crit)/6.0

    if wep['strength']>target['toughness']:
        if wep['strength']>=2*target['toughness']:
            p_wound = 5/6.0
        else: p_wound = 4/6.0
    elif wep['strength']<target['toughness']:
        if 2*wep['strength']<=target['toughness']:
            p_wound = 1/6.0
        else: p_wound = 2/6.0
    else: p_wound = 3/6.0

    p_wound = max(p_wcrit,p_wound)
    
    if verbose: print("Wound",p_wound,p_wcrit)

    # Prob to not save
    save = min(target['invuln'] or 10,target['save']-wep['AP'])
    p_nsave = 1.0 - max(1,min(6,7-save))/6.0

    if 'devastating wounds' in wep['kws']: # Devastating wounds
        p_nsave = ((p_wound-p_wcrit)*p_nsave + p_wcrit)/p_wound

    if verbose: print("Save",p_nsave)

    # Total probability
    if 'lethal hits' in wep['kws']:
        p_dam = p_hcrit*p_nsave + (p_hit-p_hcrit)*p_wound*p_nsave
    else:
        p_dam = p_hit*p_wound*p_nsave

    if verbose: print("Total prob", p_dam)

    return p_dam

In [12]:
# Return suffix of the kw if found, and None otherwise
def find_kw(kw,kws):
    sus = None
    for k in wep['kws']:
        if k.startswith(kw):
            sus = k[len(kw):].strip()
    return sus

In [ ]:
# Wrapper around atk_success_prob that handles sustained hits
def atk_success_dist(wep,target):
   
    # Find number of sustained hits
    sus = find_kw('sustained hits',wep['kws'])
   
    # Handle the easy case (no sustained hits)
    if not sus:
        p = atk_success_prob(wep,target)
        return { 1: p, 0: (1-p) }
    
    # Sustained hits:
    sus = int(sus) if sus else 0
    
    p_hit, p_hcrit = get_hit_probs(wep)
    pc, pn = atk_success_prob(wep,target,True), atk_success_prob(wep,target,False)

    p = pn*(1-p_hcrit)
    dd = { 1: pc, 0: {1-p} }

    normal = { 1: pn, 0: (1-pn) }

    crit = { 1: pc, 0: (1-pc) }
    for _ in range(sus):
        crit = convolve(crit,normal)

    normal = mult_ddist_probs(normal,p_hit-p_hcrit)
    crit = mult_ddist_probs(crit,p_hcrit)

    total =  add_ddist(normal, crit)
    total[0] += 1.0-p_hit

    return total

atk_success_dist(wep,target)


In [17]:
# Create res as weighted sum of repeated convolutions with weights given by b_dd and repeated self-convolutons of r_dd
def dd_over_dd(b_dd,r_dd,**argv):
    cur_d,res_d = {0: 1}, {0: b_dd.get(0,0.0)}
    for i in range(1,dd_max(b_dd)+1):
        cur_d = convolve(cur_d,r_dd,**argv)
        if i in b_dd:
            res_d = add_ddist(res_d,mult_ddist_probs(cur_d,b_dd[i]))
    return res_d

In [ ]:
def successful_atk_dist(wep,target):

    # Attack number dist
    an_d = dd_from_str(wep['attacks'])

    # Todo: rapid fire dX, blast
    added_attacks = 0
    if added_attacks!=0:
        an_d = convolve(an_d,{added_attacks:1})

    # Attack successes dist
    as_d = atk_success_dist(wep,target)

    # Create res as weighted sum of repeated convolutions
    res_d = dd_over_dd(an_d,as_d)

    return res_d

successful_atk_dist(wep,target)

In [ ]:
# Final end-to-end calculation for a weapon
def dam_dist(wep,target,n=1):

    # Successful attack dist
    sa_d = successful_atk_dist(wep,target)

    # Single damage dist
    # TODO: Melta + halving and reducing damage added here
    sd_d = single_dam_dist(wep['damage'],target['wounds'],fnp=target.get('fnp'))

    # Create res as weighted sum of repeated convolutions
    unit_d = dd_over_dd(sa_d,sd_d,n_wounds=target['wounds'])

    res_d = unit_d
    for _ in range(1,n):
        res_d = dd_prune(convolve(res_d,unit_d),1e-3)


    return res_d

dam_dist(wep,target)

In [20]:
import timeit

In [ ]:
%%timeit
dam_dist(wep,target,20)

# Read in unit info

In [34]:
import pandas as pd
files = ['Factions','Datasheets','Datasheets_abilities','Datasheets_keywords','Datasheets_models','Datasheets_wargear','Datasheets_unit_composition','Abilities','Last_update']
dfs = {}
for f in files:
    dfs[f] = pd.read_csv('datafiles/'+f+'.csv',sep='|')

In [30]:
# Remove link tags from wargear descriptions
dfs['Datasheets_wargear']['description'] = dfs['Datasheets_wargear']['description'].str.replace('<[^>]*>','',regex=True).str.lower()

def wargear_to_weapon(wg):
    return {
        'name': wg['name'],
        'type': wg['type'],
        'range': wg['range'],
        'attacks': wg['A'], 
        'bsws': int(str(wg['BS_WS']).strip('+')) if str(wg['BS_WS'])!='nan' else 0,
        'strength': int(wg['S']) if 'D' not in wg['S'] else 7, # TODO: Hack for now as it is 2D6 twice and 6+D6 once
        'AP': int(wg['AP']), 
        'damage': wg['D'],
        'kws': [ kw.strip() for kw in re.split(r',|\.',str(wg['description']))],
        'simplified': 'D' in str(wg['S']) # Warn if something here was simplified 
    }

def model_to_target(m):
    return { 'name': m['name'], 'toughness': int(m['T'].strip('*')), 
            'save': int(m['Sv'].strip('+')), 
            'invuln': int(m['inv_sv'].strip('*')) if str(m['inv_sv'])!='-' else None, 
            'wounds': int(m['W']),
            'simplified': '*' in m['inv_sv'] or '*' in m['T'] # Warn if something here was simplified 
    }

In [31]:
# Simplify abilities down to a dict of lists
ability_dict = pd.Series(dfs['Abilities']['name'].values,index=dfs['Abilities']['id']).to_dict()
df = dfs['Datasheets_abilities']
df.loc[~df['ability_id'].isna(),'name'] = df.loc[~df['ability_id'].isna(),'ability_id'].replace(ability_dict)
df.loc[~df['parameter'].isna(),'name'] += ' ' + df.loc[~df['parameter'].isna(),'parameter']
abilities = df.groupby('datasheet_id')['name'].apply(list)

keywords = dfs['Datasheets_keywords'].groupby('datasheet_id')['keyword'].apply(list)

In [32]:
rd = {}

dsgb = dfs['Datasheets'].groupby('faction_id')
dsmgb = dfs['Datasheets_models'].groupby('datasheet_id')
dswggb = dfs['Datasheets_wargear'].groupby('datasheet_id')
for i, f in dfs['Factions'].iterrows():

    dsl = [ { 'name': ds['name'], 'id': ds['id'] } for _,ds in dsgb.get_group(f['id']).iterrows() ]
    rd[f['name']] = dsl

    for ds in dsl:
        if ds['id'] not in dswggb.groups: continue
        ds['weapons'] = list(dswggb.get_group(ds['id']).apply(wargear_to_weapon,axis=1))

        if ds['id'] not in dsmgb.groups: continue
        ds['models'] = list(dsmgb.get_group(ds['id']).apply(model_to_target,axis=1))

        for m in ds['models']:
            m['abilities'] = abilities[ds['id']]
            m['kws'] = keywords[ds['id']]
        

In [ ]:
unit = rd['Death Guard'][10]
wep,target = unit['weapons'][4],unit['models'][0]
wep,target,dam_dist(wep,target)